In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer
from sklearn.svm import SVC

import pandas as pd
import numpy as np
import pickle

In [2]:
PATH = "/home/parazit/ml_virus_host/v2.0/"
PATH_FEATURES = PATH + "v3.0/baseline/htp/features/"
PATH_SAMPLE_IDS = PATH + "sample_ids/genomes_fragments_connected/"
PATH_DATA = PATH + "data/"

In [3]:
meta_df_genomes = pd.read_csv(PATH_DATA+"data_table.tsv", sep="\t", index_col=0)
meta_df_800 = pd.read_csv(PATH_DATA+"data_table_800.tsv", sep="\t", index_col=0)
meta_df_400 = pd.read_csv(PATH_DATA+"data_table_400.tsv", sep="\t", index_col=0)

In [4]:
sample_ids_genomes = pickle.load(open(PATH_SAMPLE_IDS+"train_val_test_genomes.pkl", "rb"))
sample_ids_800 = pickle.load(open(PATH_SAMPLE_IDS+"train_val_test_800.pkl", "rb"))
sample_ids_400 = pickle.load(open(PATH_SAMPLE_IDS+"train_val_test_400.pkl", "rb"))

Функция подбора параметров моделей

In [5]:
def find_best_model(X, y, clf, param_grid):
    model = GridSearchCV(estimator = clf, param_grid = param_grid, cv=5, verbose=10, scoring="f1_weighted", n_jobs = -1)
    model.fit(X,y)
    print('Best weighted f1-score of Grid search: {}'.format(model.best_score_))
    print("Best model parameters of Grid search: {}".format(model.best_params_))
    return model.best_estimator_

Функция выделения X (признаки) и y (целевая переменная)

In [6]:
def get_X_y(name, meta_df):
    
    df = pd.read_csv(PATH_FEATURES+name+"_features.tsv", sep="\t", index_col=0)
    y = meta_df.loc[df.index].host.values
    X = StandardScaler().fit_transform(df.values) # шкалирвоание проводится внутри функции
    
    return X, y

# Обучение на геномах, тестирование на выборках геномов и фрагментов

## Логистическая регрессия

Перебор параметров

In [7]:
param_grid = {
                "penalty":["l1"], 
                "C": [1, 10, 100, 1000],
                "solver": ["liblinear", "saga"],
                "class_weight" : ["balanced"],
                "max_iter": [300]
             }

Выделям тренировочную выборку, обучаем логистическую регрессию

In [8]:
X_train, y_train = get_X_y("train_genomes", meta_df_genomes)

best_model = find_best_model(X_train, y_train, LogisticRegression(), param_grid)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/home/parazit/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/parazit/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/parazit/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/parazit/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/parazit/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/parazit/anaconda3/lib/python3.9/site-packages/sklearn/linea

Best weighted f1-score of Grid search: 0.9150178200949194
Best model parameters of Grid search: {'C': 1, 'class_weight': 'balanced', 'max_iter': 300, 'penalty': 'l1', 'solver': 'liblinear'}


Тестируем на геномах

In [9]:
X_test, y_test = get_X_y("test_genomes", meta_df_genomes) # шкалирвоание проводится внутри функции

print(classification_report(y_test, best_model.predict(X_test)))

               precision    recall  f1-score   support

      Insecta       0.55      0.64      0.60        95
     Mammalia       0.83      0.85      0.84       160
Viridiplantae       0.62      0.48      0.54        83

     accuracy                           0.70       338
    macro avg       0.67      0.66      0.66       338
 weighted avg       0.70      0.70      0.70       338



Тестируем на фрагментах длины 800 нуклеотидов

In [10]:
X_test, y_test = get_X_y("test_800", meta_df_800)

print(classification_report(y_test, best_model.predict(X_test)))

               precision    recall  f1-score   support

      Insecta       0.47      0.49      0.48       190
     Mammalia       0.73      0.77      0.75       320
Viridiplantae       0.42      0.36      0.39       166

     accuracy                           0.59       676
    macro avg       0.54      0.54      0.54       676
 weighted avg       0.58      0.59      0.59       676



Тестируем на фрагментах длины 400 нуклеотидов

In [11]:
X_test, y_test = get_X_y("test_400", meta_df_400)

print(classification_report(y_test, best_model.predict(X_test)))

               precision    recall  f1-score   support

      Insecta       0.43      0.41      0.42       190
     Mammalia       0.69      0.71      0.70       320
Viridiplantae       0.37      0.37      0.37       166

     accuracy                           0.54       676
    macro avg       0.50      0.50      0.50       676
 weighted avg       0.54      0.54      0.54       676



Взвешенная F1-мера полногеномной LR Host Taxon Predictor:

На геномах 0.70, на фрагментах 800 нуклеотидов 0.58, на фрагментах 400 нуклеотидов 0.54

## Метод опорных векторов

Параметры регуляризации (С) соответствуют таковым в референсном исследовании Host Taxon Predictor

In [13]:
host_c_svc = {"Insecta": 0.03125, "Mammalia": 0.03125, "Viridiplantae": 0.25}

Выделям тренировочную выборку, обучаем 3 бинарных классификатора SVC

In [14]:
X_train, y_train = get_X_y("train_genomes", meta_df_genomes)

svc_genome_models = {}
y_dict_svc = dict(zip(["Insecta", "Mammalia", "Viridiplantae"], LabelBinarizer().fit(y_train).transform(y_train).T))

for host in host_c_svc.keys():
    
    svc_genome_models[host] = \
                SVC(kernel = 'linear', probability = True, C = host_c_svc[host]).fit(X_train, y_dict_svc[host])

Оцениваем качество мультиклассовой классификации на основе бинарных вероятностей

In [15]:
def SVC_multiclassification(models, X_test, y_test, print_binary = False):
    
    y_proba = np.zeros(shape = y_test.shape)
    y_dict_svc = dict(zip(["Insecta", "Mammalia", "Viridiplantae"], LabelBinarizer().fit(y_test).transform(y_test).T))
    
    for host in y_dict_svc.keys():
        y_proba = np.vstack((y_proba, models[host].predict_proba(X_test)[:,1]))
        if print_binary:
            print(classification_report(y_dict_svc[host], models[host].predict(X_test), target_names = ['Others', host]))

    y_pred  = pd.Series(np.argmax((y_proba[1:]/y_proba[1:].sum(axis=0)), axis=0)).map({0: "Insecta", 1: "Mammalia", 2: "Viridiplantae"}).values
    print(classification_report(y_test, y_pred, target_names = ["Insecta", "Mammalia", "Viridiplantae"], zero_division=1))
    
    return

Тестируем на полных геномах


In [16]:
X_test, y_test = get_X_y("test_genomes", meta_df_genomes)

SVC_multiclassification(svc_genome_models, X_test, y_test)

               precision    recall  f1-score   support

      Insecta       0.60      0.61      0.60        95
     Mammalia       0.81      0.89      0.85       160
Viridiplantae       0.69      0.54      0.61        83

     accuracy                           0.73       338
    macro avg       0.70      0.68      0.69       338
 weighted avg       0.72      0.73      0.72       338



Тестируем на фрагментах 800 нуклеотидов

In [17]:
X_test, y_test = get_X_y("test_800", meta_df_800)

SVC_multiclassification(svc_genome_models, X_test, y_test)

               precision    recall  f1-score   support

      Insecta       0.47      0.48      0.47       190
     Mammalia       0.72      0.80      0.76       320
Viridiplantae       0.47      0.36      0.41       166

     accuracy                           0.60       676
    macro avg       0.55      0.55      0.55       676
 weighted avg       0.59      0.60      0.59       676



Тестируем на фрагментах 400 нуклеотидов

In [18]:
X_test, y_test = get_X_y("test_400", meta_df_400)

SVC_multiclassification(svc_genome_models, X_test, y_test)

               precision    recall  f1-score   support

      Insecta       0.43      0.40      0.41       190
     Mammalia       0.67      0.72      0.70       320
Viridiplantae       0.38      0.35      0.36       166

     accuracy                           0.54       676
    macro avg       0.49      0.49      0.49       676
 weighted avg       0.53      0.54      0.54       676



Взвешенная F1-мера полногеномных SVC Host Taxon Predictor:

На геномах 0.71, на фрагментах 800 нуклеотидов 0.59, на фрагментах 400 нуклеотидов 0.54
